<a href="https://colab.research.google.com/github/OmarZohir/Unet-MRI/blob/main/FastMRI_Unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%matplotlib inline
import h5py
import numpy as np
from matplotlib import pyplot as plt
import glob
from PIL import Image
import fastmri
from fastmri.data import transforms as T
from fastmri.data.subsample import RandomMaskFunc


SyntaxError: ignored

In [ ]:
file_names = glob.glob('*.h5')
#hf = h5py.File(file_name)
file_names

['file1000071.h5',
 'file1000073.h5',
 'file1000153.h5',
 'file1000201.h5',
 'file1000314.h5',
 'file1000325.h5',
 'file1000328.h5',
 'file1000350.h5',
 'file1000356.h5',
 'file1000432.h5',
 'file1000480.h5',
 'file1000537.h5',
 'file1000552.h5',
 'file1000555.h5',
 'file1000647.h5',
 'file1000660.h5',
 'file1000758.h5',
 'file1000810.h5',
 'file1000842.h5',
 'file1000858.h5',
 'file1000871.h5',
 'file1000885.h5',
 'file1000891.h5']

In [ ]:
# To save the files in directories

for file in file_names:
    hf = h5py.File(file)
    original_images = hf['reconstruction_esc'][()]
    for i in range(original_images.shape[0]):
        data = original_images[i]
        rescaled = (255.0 / data.max() * (data - data.min())).astype(np.uint8)
        im = Image.fromarray(rescaled)
        im.save("output/" + file[:-3] + "_" + str(i) + "recon.jpg")

for file in file_names:
    hf = h5py.File(file)
    original_images = hf['kspace'][()]
    for i in range(original_images.shape[0]):
        slice_kspace2 = T.to_tensor(original_images[i])
        mask_func = RandomMaskFunc(center_fractions=[0.04], accelerations=[8])
        masked_kspace, mask = T.apply_mask(slice_kspace2, mask_func)
        sampled_image = fastmri.ifft2c(masked_kspace)
        sampled_image_abs = fastmri.complex_abs(sampled_image).numpy()
        data = sampled_image_abs[320-160:320+160, 184-160:184+160]
        rescaled = (255.0 / data.max() * (data - data.min())).astype(np.uint8)
        im = Image.fromarray(rescaled)
        im.save("input/" + file[:-3] + "_" + str(i) + "_kspace.jpg")





<ipython-input-4-5d645e11812f>:4: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  hf = h5py.File(file)
<ipython-input-4-5d645e11812f>:13: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  hf = h5py.File(file)


In [ ]:
# To save the numpy arrays with the values
count = 0
full_output = np.zeros((800,320,320))
for file in file_names:
    hf = h5py.File(file)
    original_images = hf['reconstruction_esc'][()]
    for i in range(1, original_images.shape[0]):
        data = original_images[i]
        rescaled = (255.0 / data.max() * (data - data.min())).astype(np.uint8)
        full_output[count,:,:] = rescaled
        count = count + 1

outputs = full_output[0:count]
outputs = outputs[..., np.newaxis]

count = 0
full_input = np.zeros((800,320,320))

for file in file_names:
    hf = h5py.File(file)
    original_images = hf['kspace'][()]
    for i in range(1, original_images.shape[0]):
        slice_kspace2 = T.to_tensor(original_images[i])
        mask_func = RandomMaskFunc(center_fractions=[0.04], accelerations=[8])
        masked_kspace, mask = T.apply_mask(slice_kspace2, mask_func)
        sampled_image = fastmri.ifft2c(masked_kspace)
        sampled_image_abs = fastmri.complex_abs(sampled_image).numpy()
        data = sampled_image_abs[320-160:320+160, 184-160:184+160]
        rescaled = (255.0 / data.max() * (data - data.min())).astype(np.uint8)
        full_input[count,:,:] = rescaled
        count = count + 1

inputs = full_input[0:count]
inputs = inputs[..., np.newaxis]

<ipython-input-5-459e75b7959d>:5: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  hf = h5py.File(file)
<ipython-input-5-459e75b7959d>:20: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  hf = h5py.File(file)


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from keras.models import Model, load_model
from keras.layers import Input
from keras import optimizers
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import tensorflow as tf
import os

# UNETs
from keras_unet.models import satellite_unet
from keras_unet.models import custom_unet

-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------


In [ ]:
def batch_ssim(y_true, y_pred):
    return tf.reduce_mean(tf.image.ssim(y_true, y_pred, max_val=1.0))


def batch_msssim(y_true, y_pred):
    return tf.reduce_mean(tf.image.ssim_multiscale(y_true, y_pred, max_val=1.0))


def batch_psnr(y_true, y_pred):
    return tf.reduce_mean(tf.image.psnr(y_true, y_pred, max_val=1.0))


def batch_nmse(y_true, y_pred):
    return tf.divide(tf.reduce_sum(tf.keras.metrics.mean_squared_error(y_true, y_pred)), tf.reduce_sum(y_true ** 2))


def loss_func_ssim(labels, predictions):
        return 1 - tf.image.ssim(labels, predictions, max_val=15.0)

In [ ]:
model=None

In [ ]:
def u1():
    inputs = Input((320, 320, 1))
    #s = Lambda(lambda x: x / 255) (inputs)

    c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (inputs)
    c1 = Dropout(0.1) (c1)
    c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
    c2 = Dropout(0.1) (c2)
    c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
    c3 = Dropout(0.2) (c3)
    c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
    c4 = Dropout(0.2) (c4)
    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

    c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
    c5 = Dropout(0.3) (c5)
    c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
    c6 = Dropout(0.2) (c6)
    c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
    c7 = Dropout(0.2) (c7)
    c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
    c8 = Dropout(0.1) (c8)
    c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
    c9 = Dropout(0.1) (c9)
    c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer=optimizers.Adam(lr=0.001), loss='mae', metrics=[batch_ssim, batch_psnr, batch_msssim, batch_nmse])
    return model

In [ ]:
model = None

def u2():
    model = satellite_unet(input_shape=(320, 320, 1))
    model.compile(optimizer=optimizers.Adam(lr=0.0001), loss='mae', metrics=[batch_ssim, batch_psnr, batch_msssim, batch_nmse])
    return model

def u3():
    model = custom_unet(
    (320, 320, 1),
    use_batch_norm=False,
    num_classes=1,
    filters=64,
    dropout=0.2,
    output_activation='sigmoid'
)

    model.compile(optimizer=optimizers.Adam(lr=0.001), loss=loss_func_ssim, metrics=[batch_ssim, batch_psnr, batch_msssim, batch_nmse])
    return model

model = u2()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 320, 320, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 320, 320, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 320, 320, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 320, 320, 64) 36928       batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
inputs.shape, outputs.shape

((763, 320, 320, 1), (763, 320, 320, 1))

In [ ]:
h = model.fit(inputs, outputs, validation_split=0.05, batch_size=2, epochs=20)